In [1]:
TRAIN_PATH = 'train_amex/historical_user_logs.csv'

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
dataset =  pd.read_csv(TRAIN_PATH)
print(dataset.columns)

Index(['DateTime', 'user_id', 'product', 'action'], dtype='object')


In [27]:
val_frame = dataset[['DateTime','user_id','product','action']]

In [28]:
print(val_frame.head)

<bound method NDFrame.head of                   DateTime  user_id product action
0         2017-05-28 15:44      704       B   view
1         2017-05-29 07:08   499679       F   view
2         2017-05-29 07:10   499679       G   view
3         2017-05-29 07:10   499679       G   view
4         2017-05-29 07:10   499679       G   view
5         2017-05-29 07:10   499679       G   view
6         2017-05-29 07:10   499679       G   view
7         2017-05-29 07:10   499679       G   view
8         2017-05-29 07:10   499679       G   view
9         2017-05-29 07:11   499679       G   view
10        2017-05-29 07:11   499679       G   view
11        2017-05-29 07:11   499679       G   view
12        2017-05-29 07:11   499679       G   view
13        2017-05-29 07:11   499679       A   view
14        2017-05-29 07:11   499679       A   view
15        2017-05-29 07:11   499679       A   view
16        2017-05-29 07:11   499679       A   view
17        2017-05-29 07:11   499679       A   view
1

In [29]:
val_temp = val_frame
val_temp['DateTime'] = val_temp['DateTime'].apply(lambda x: (str(x)[11:13]))
val_temp['DateTime'] = pd.to_numeric(val_temp['DateTime'])

In [30]:
print(len(val_temp['user_id'].unique()))
print(len(val_temp))

171309
24287534


In [31]:
print(val_temp['DateTime'].isna().sum())
print(val_temp['user_id'].isna().sum())
print(val_temp['product'].isna().sum())
print(val_temp['action'].isna().sum())

0
0
0
0


In [32]:
len(val_temp)

24287534

In [11]:
val_temp.user_id = val_temp.user_id.apply(str)

In [12]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [16]:
val_temp = MultiColumnLabelEncoder(columns = ['user_id']).fit_transform(val_temp)
val_temp = MultiColumnLabelEncoder(columns = ['product']).fit_transform(val_temp)
val_temp = MultiColumnLabelEncoder(columns = ['action']).fit_transform(val_temp)

In [17]:
ohe = OneHotEncoder(categorical_features=[1,2,3])

val_temp = ohe.fit_transform(val_temp).toarray()

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').